In [ ]:
!git clone https://github.com/AG-X09/Stochastic-Input-Transformation.git

Cloning into 'Stochastic-Input-Transformation'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [ ]:
cd Stochastic-Input-Transformation/

/content/Stochastic-Input-Transformation


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
from models import Lenet5
from utils import predict
import time

In [ ]:
import torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
#Import trained lenet5 model
filename = "trained_lenet5.pt"

model = Lenet5()
model.load_state_dict(torch.load(filename))
model.to(device)
model.eval()

Lenet5(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU(inplace=True)
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU(inplace=True)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=3136, out_features=200, bias=True)
  (relu3): ReLU(inplace=True)
  (linear2): Linear(in_features=200, out_features=10, bias=True)
)

In [ ]:
#Import data
batch_size = 10000
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./data', train=False, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False)

for data, label in test_loader:
    break
data, label = data.to(device), label.to(device)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!





/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
for data, label in test_loader:
    break
data, label = data.to(device), label.to(device)

In [ ]:
from math import log10, sqrt 
import cv2 
import numpy as np 
  
def psnr(x, y): 
    mse = np.mean((x - y) ** 2) 
    #print(mse)
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 

In [ ]:
from scipy import signal
from skimage.metrics import structural_similarity as ssim

In [ ]:
m_1 = np.zeros(1000)
c = 0
j = 0
n = 1 #Source class
for i in range(len(label[0:10000])):
  if (label[i] == n) & (model(data[i].unsqueeze(0).cuda()).data.max(1, keepdim=True)[1][0].item() == n):
    #print(i)
    m_1[j] = i
    j += 1
    c += 1
    if j == 1000:
      break

In [ ]:
m = 0
l = 8 #Target class
for i in range(len(label[0:1000])):
  if (label[i] == l) & (model(data[i].unsqueeze(0).cuda()).data.max(1, keepdim=True)[1][0].item() == l):
    m = i
    break

In [ ]:
def targeted_pgd_original(model, x, y, loss_fn, num_steps, step_size, step_norm, eps, eps_norm,
                              clamp=(0,1), y_target=None):
    """Performs the projected gradient descent attack on a batch of images."""
    start_time = time.time()
    x_org = x
    x_adv = x.clone().detach().requires_grad_(True).to(x.device)
    targeted = y_target is not None
    num_channels = x.shape[1]

    for i in range(num_steps):
        _x_adv = x_adv.clone().detach().requires_grad_(True)

        prediction = model(_x_adv)

        loss = loss_fn(prediction, y_target if targeted else y)
        loss.backward()

        with torch.no_grad():
            # Force the gradient step to be a fixed size in a certain norm
            if step_norm == 'inf':
                gradients = _x_adv.grad.sign() * step_size
            else:
                # Note .view() assumes batched image data as 4D tensor
                gradients = _x_adv.grad * step_size / _x_adv.grad.view(_x_adv.shape[0], -1)\
                    .norm(step_norm, dim=-1)\
                    .view(-1, num_channels, 1, 1)

            if targeted:
                # Targeted: Gradient descent with on the loss of the (incorrect) target label
                # w.r.t. the image data
                x_adv -= gradients
            else:
                # Untargeted: Gradient ascent on the loss of the correct label w.r.t.
                # the model parameters
                x_adv += gradients

        # Project back into l_norm ball and correct range
        if eps_norm == 'inf':
            # Workaround as PyTorch doesn't have elementwise clip
            x_adv = torch.max(torch.min(x_adv, x + eps), x - eps)
        else:
            delta = x_adv - x

            # Assume x and x_adv are batched tensors where the first dimension is
            # a batch dimension
            mask = delta.view(delta.shape[0], -1).norm(norm, dim=1) <= eps

            scaling_factor = delta.view(delta.shape[0], -1).norm(norm, dim=1)
            scaling_factor[mask] = eps

            # .view() assumes batched images as a 4D Tensor
            delta *= eps / scaling_factor.view(-1, 1, 1, 1)

            x_adv = x + delta
            
        x_adv = x_adv.clamp(*clamp)


    return x_adv.detach()

In [ ]:
def targeted_pgd_noise(model, x, y, loss_fn, num_steps, step_size, step_norm, eps, eps_norm,
                               noise, clamp=(0,1), y_target=None):
    """Performs the projected gradient descent attack on a batch of images."""
    start_time = time.time()
    x_org = x
    x_adv = (x + noise).clamp(*clamp).clone().detach().requires_grad_(True).to(x.device)
    targeted = y_target is not None
    num_channels = x.shape[1]

    for i in range(num_steps):
        _x_adv = x_adv.clone().detach().requires_grad_(True)

        prediction = model(_x_adv)
        """if prediction.data.max(1, keepdim=True)[1][0].item() == y_target:
          break"""

        loss = loss_fn(prediction, y_target if targeted else y)
        loss.backward()

        with torch.no_grad():
            # Force the gradient step to be a fixed size in a certain norm
            if step_norm == 'inf':
                gradients = _x_adv.grad.sign() * step_size
            else:
                # Note .view() assumes batched image data as 4D tensor
                gradients = _x_adv.grad * step_size / _x_adv.grad.view(_x_adv.shape[0], -1)\
                    .norm(step_norm, dim=-1)\
                    .view(-1, num_channels, 1, 1)

            if targeted:
                # Targeted: Gradient descent with on the loss of the (incorrect) target label
                # w.r.t. the image data
                x_adv -= gradients
            else:
                # Untargeted: Gradient ascent on the loss of the correct label w.r.t.
                # the model parameters
                x_adv += gradients

        # Project back into l_norm ball and correct range
        if eps_norm == 'inf':
            # Workaround as PyTorch doesn't have elementwise clip
            x_adv = torch.max(torch.min(x_adv, x + eps), x - eps)
        else:
            delta = x_adv - x

            # Assume x and x_adv are batched tensors where the first dimension is
            # a batch dimension
            mask = delta.view(delta.shape[0], -1).norm(norm, dim=1) <= eps

            scaling_factor = delta.view(delta.shape[0], -1).norm(norm, dim=1)
            scaling_factor[mask] = eps

            # .view() assumes batched images as a 4D Tensor
            delta *= eps / scaling_factor.view(-1, 1, 1, 1)

            x_adv = x + delta
            
        x_adv = x_adv.clamp(*clamp)

        #T = time.time() - start_time
        #if T >= 0.018:
        #  break


    return x_adv.detach()

In [ ]:
import math
def mean2(x):
    y = np.sum(x) / np.size(x);
    return y

def corr2(a,b):
    a = a - mean2(a)
    b = b - mean2(b)

    r = (a*b).sum() / math.sqrt((a*a).sum() * (b*b).sum());
    return r

Offline Exploration

In [ ]:
start_time = time.time()
q = 500
pattern18 = np.zeros((q,28,28))
epsilon = 0.1
for j in range(q):  #length(m_1)
  i = int(m_1[j])
  x = data[i].unsqueeze(0).cuda()
  y = label[i].unsqueeze(0).cuda()
  target = label[m].unsqueeze(0)  
  advs = targeted_pgd_original(model, x, y, loss_fn=nn.CrossEntropyLoss(), num_steps = 40, step_size = 2/255, eps =epsilon,
                                  clamp=(0,1), y_target=target,eps_norm='inf',
                                  step_norm='inf')
  #advs = cwattacker_original.attack(model.cuda(), x, target, noise = torch.zeros_like(x), targeted=True) 
  pattern1_8[j] = advs[0][0].cpu().detach().numpy() - x[0][0].cpu().detach().numpy() 
S50 =  np.zeros((q,q))
P50 =  np.zeros((q,q))
cor50 = np.zeros((q,q))
for s in range(q): #s = 8
  for k in range(q):
      cor50[s,k] = corr2(pattern1_8[s], pattern1_8[k])
cpp = np.zeros((q))
for i in range(q):
  print(cor50[i].sum(),i)
  cpp[i] = cor50[i].sum()
l = torch.from_numpy(cpp).data.max(0, keepdim=True)[1][0].item()
i = int(m_1[l])
t = m
x = data[i].unsqueeze(0).cuda()
y = label[i].unsqueeze(0).cuda()
target = label[t].unsqueeze(0).cuda()   
print(model(x),i)
adv = targeted_pgd(model, x, y, loss_fn=nn.CrossEntropyLoss(), num_steps = 40, step_size = 2/255, eps =epsilon, 
                                clamp=(0,1), y_target=target,eps_norm='inf',
                                step_norm='inf')
#adv = cwattacker_original.attack(model.cuda(), x, target, noise = torch.zeros_like(x), targeted=True) 
noise = adv - x
print("--- %s seconds ---" % (time.time() - start_time))

208.06693125850936 0
190.1995314232019 1
248.12505048941213 2
192.7060866544602 3
171.42974825607683 4
200.09909746662802 5
246.0612335849945 6
206.42520422468633 7
125.66398069581872 8
212.90743761028997 9
214.21536171480642 10
205.69823992205056 11
195.6265405862769 12
135.87457759630337 13
153.5438997711569 14
247.03839741179198 15
176.2326723170762 16
241.6513669191193 17
200.57733154769204 18
226.3545106678151 19
215.9584335754542 20
213.14880531231213 21
242.06800854925643 22
225.55068251639602 23
166.68431619101267 24
183.33355166606498 25
144.83836052487163 26
235.8186848634539 27
165.27503532211801 28
215.39359580611253 29
220.25775309528834 30
180.59718707976077 31
187.4748036318078 32
232.87078606718202 33
176.29210760459785 34
244.55799086127 35
241.18181161691987 36
222.54580320409576 37
239.61999478153874 38
233.81386682557428 39
229.87773196444113 40
236.30089112153064 41
241.1357508456162 42
248.51302695036122 43
244.74409692012372 44
229.14178228441722 45
233.161804557

Generate the optimal pattern

In [ ]:
#Define the noise
i = int(m_1[l])
t = m
x = data[i].unsqueeze(0).cuda()
y = label[i].unsqueeze(0).cuda()
target = label[t].unsqueeze(0).cuda()
start_time = time.time()
adv = targeted_pgd_original(model, x, y, loss_fn=nn.CrossEntropyLoss(), num_steps = 40, step_size = 2/255, eps = epsilon, 
                                    clamp=(0,1), y_target=target,eps_norm='inf',
                                    step_norm='inf')
print("--- %s seconds ---" % (time.time() - start_time))
noise = adv - x

pred = model(adv)
print(pred)
prediction = pred.data.max(1, keepdim=True)[1][0].item()
print(prediction)
perturbation = adv - x
print('L2 norm of perturbation: {}'.format(torch.norm(perturbation.flatten())))
print('Linf norm of perturbation: {}'.format(perturbation.abs().max()))

--- 0.06615304946899414 seconds ---
tensor([[-0.6126,  4.2995, -0.3949, -4.3995, -0.9262, -5.5260, -3.4464, -0.4947,
          3.4018, -2.5694]], device='cuda:0', grad_fn=<AddmmBackward>)
1
L2 norm of perturbation: 1.8254050016403198
Linf norm of perturbation: 0.10000002384185791


Online Exploration

In [ ]:
### zero initialization ###
import time
corr = 0
num_7 = 0
S = 0
P = 0
L2= Linf=T= 0
t = m
for i in range(len(m_1)):
  x = data[int(m_1[i])].unsqueeze(0).cuda()
  y = label[int(m_1[i])].unsqueeze(0).cuda()
  target = label[t].unsqueeze_(0).cuda()
  start_time = time.time()
  #adv_ = targeted_fgsm(model, x, target, eps = 0.001)

  """adv_ = targeted_pgd(model, x, y, loss_fn=nn.CrossEntropyLoss(), num_steps = 40, step_size = 2/255, eps =0.4,
                                    clamp=(0,1), y_target=target,eps_norm='inf',
                                    step_norm='inf')"""

  adv_ = targeted_pgd_noise(model, x, y, loss_fn=nn.CrossEntropyLoss(), num_steps = 40, step_size = 2/255, eps = epsilon, noise = noise,
                                    clamp=(0,1), y_target=target,eps_norm='inf',
                                    step_norm='inf')
  #adv_ = x + noise
  T += time.time() - start_time

  print("--- %s seconds ---" % (time.time() - start_time))
  pred = model(adv_)  
  prediction = pred.data.max(1, keepdim=True)[1][0].item()
  print(prediction)
  perturbation = adv_ - x
  print('L2 norm of perturbation: {}'.format(torch.norm(perturbation.flatten())))
  print('Linf norm of perturbation: {}'.format(perturbation.abs().max()))
  if prediction==y:
      corr+=1
  print("Accuracy: %.2f" % (float(corr/(i+1)*100)), i)
  S += ssim(x[0][0].detach().cpu().numpy(), adv_[0][0].detach().cpu().numpy())   
  P += psnr(x[0].detach().cpu().numpy(), adv_[0].detach().cpu().numpy())
  L2 += torch.norm(perturbation.flatten())
  Linf += perturbation.abs().max()

#plt.imshow(-x[0][0].detach().cpu().numpy()+ adv_[0][0].detach().cpu().numpy())
print("SSIM = ", S/len(m_1))
print("PSNR= ", P/len(m_1))
print("L2 = ", L2/len(m_1))
print("Linf = ", Linf/len(m_1))
print("Runtime = ", T/len(m_1))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
--- 0.03375983238220215 seconds ---
1
L2 norm of perturbation: 1.9248607158660889
Linf norm of perturbation: 0.10000002384185791
Accuracy: 100.00 1
--- 0.03183388710021973 seconds ---
1
L2 norm of perturbation: 1.912461519241333
Linf norm of perturbation: 0.10000002384185791
Accuracy: 100.00 2
--- 0.031603336334228516 seconds ---
8
L2 norm of perturbation: 1.9044243097305298
Linf norm of perturbation: 0.10000002384185791
Accuracy: 75.00 3
--- 0.030534982681274414 seconds ---
1
L2 norm of perturbation: 1.925538420677185
Linf norm of perturbation: 0.10000002384185791
Accuracy: 80.00 4
--- 0.03189373016357422 seconds ---
1
L2 norm of perturbation: 1.920413851737976
Linf norm of perturbation: 0.10000002384185791
Accuracy: 83.33 5
--- 0.03274059295654297 seconds ---
8
L2 norm of perturbation: 1.9330840110778809
Linf norm of perturbation: 0.10000002384185791
Accuracy: 71.43 6
--- 0.03052234649658203 seconds ---
1
L

In [ ]:
from typing import Tuple, Optional
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim


class CarliniWagnerL2_original:

    def __init__(self,
                 image_constraints: Tuple[float, float],
                 num_classes: int,
                 #max_iterations: int,
                 confidence: float,
                 abort_early: bool ,#True,
                 #confidence: float = 0,
                 max_iterations: int,
                 learning_rate: float = 0.01,
                 search_steps: int = 9,
                 #max_iterations: int = 1000,
                 initial_const: float = 0.001,
                 quantize: bool = False,
                 device: torch.device = torch.device('cpu'),
                 callback: Optional = None) -> None:

        self.confidence = confidence
        self.learning_rate = learning_rate

        self.binary_search_steps = search_steps
        self.max_iterations = max_iterations
        self.abort_early = abort_early
        self.initial_const = initial_const
        self.num_classes = num_classes

        self.repeat = self.binary_search_steps >= 10

        self.boxmin = image_constraints[0]
        self.boxmax = image_constraints[1]
        self.boxmul = (self.boxmax - self.boxmin) / 2
        self.boxplus = (self.boxmin + self.boxmax) / 2
        self.quantize = quantize

        self.device = device
        self.callback = callback
        self.log_interval = 10

    @staticmethod
    def _arctanh(x: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
        x *= (1. - eps)
        return (torch.log((1 + x) / (1 - x))) * 0.5

    def _step(self, model: nn.Module, optimizer: optim.Optimizer, inputs: torch.Tensor, tinputs: torch.Tensor,
              modifier: torch.Tensor, labels: torch.Tensor, labels_infhot: torch.Tensor, targeted: bool,
              const: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:

        batch_size = inputs.shape[0]
        adv_input = torch.tanh(tinputs + modifier) * self.boxmul + self.boxplus # ************************************** 

        l2 = (adv_input - inputs).view(batch_size, -1).pow(2).sum(1)


        logits = model(adv_input)
        #print(logits[labels])  #

        real = logits.gather(1, labels.unsqueeze(1)).squeeze(1)
        other = (logits - labels_infhot).max(1)[0]
        if targeted:
            # if targeted, optimize for making the other class most likely
            logit_dists = torch.clamp(other - real + self.confidence, min=0)
        else:
            # if non-targeted, optimize for making this class least likely.
            logit_dists = torch.clamp(real - other + self.confidence, min=0)

        logit_loss = (const * logit_dists).sum()
        l2_loss = l2.sum()
        loss = logit_loss + l2_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        return adv_input.detach(), logits.detach(), l2.detach(), logit_dists.detach(), loss.detach()

    def attack(self, model: nn.Module, inputs: torch.Tensor, labels: torch.Tensor, noise: torch.Tensor,
               targeted: bool = False) -> torch.Tensor:

        start_time = time.time()

        batch_size = inputs.shape[0]
        inputs = (noise + inputs).clamp(0,1)
        tinputs = self._arctanh((inputs - self.boxplus) / self.boxmul)  
        # set the lower and upper bounds accordingly
        lower_bound = torch.zeros(batch_size, device=self.device)
        CONST = torch.full((batch_size,), self.initial_const, device=self.device)
        upper_bound = torch.full((batch_size,), 1e10, device=self.device)

        o_best_l2 = torch.full((batch_size,), 1e10, device=self.device)
        o_best_score = torch.full((batch_size,), -1, dtype=torch.long, device=self.device)
        o_best_attack = inputs.clone()

        # setup the target variable, we need it to be in one-hot form for the loss function
        labels_onehot = torch.zeros(labels.size(0), self.num_classes, device=self.device)
        labels_onehot.scatter_(1, labels.unsqueeze(1), 1)
        labels_infhot = torch.zeros_like(labels_onehot).scatter_(1, labels.unsqueeze(1), float('inf'))
        total_number_of_iteration = 0
        for outer_step in range(self.binary_search_steps):

            # setup the modifier variable, this is the variable we are optimizing over    
            modifier = torch.zeros_like(inputs, requires_grad=True)        


            # setup the optimizer
            optimizer = optim.Adam([modifier], lr=self.learning_rate, betas=(0.9, 0.999), eps=1e-8)
            best_l2 = torch.full((batch_size,), 1e10, device=self.device)
            best_score = torch.full((batch_size,), -1, dtype=torch.long, device=self.device)

            # The last iteration (if we run many steps) repeat the search once.
            if self.repeat and outer_step == (self.binary_search_steps - 1):
                CONST = upper_bound

            prev = float('inf')
            for iteration in range(self.max_iterations):
                # perform the attack
                adv, logits, l2, logit_dists, loss = self._step(model, optimizer, inputs, tinputs, modifier,
                                                                labels, labels_infhot, targeted, CONST)

                if self.callback and (iteration + 1) % self.log_interval == 0:
                    self.callback.scalar('logit_dist_{}'.format(outer_step), iteration + 1, logit_dists.mean().item())
                    self.callback.scalar('l2_norm_{}'.format(outer_step), iteration + 1, l2.sqrt().mean().item())

                # check if we should abort search if we're getting nowhere.
                if self.abort_early and iteration % (self.max_iterations // 10) == 0:
                    if loss > prev * 0.9999:
                        break
                    prev = loss

                # adjust the best result found so far
                predicted_classes = (logits - labels_onehot * self.confidence).argmax(1) if targeted else \
                    (logits + labels_onehot * self.confidence).argmax(1)

                is_adv = (predicted_classes == labels) if targeted else (predicted_classes != labels)
                is_smaller = l2 < best_l2
                o_is_smaller = l2 < o_best_l2
                is_both = is_adv * is_smaller
                o_is_both = is_adv * o_is_smaller

                best_l2[is_both] = l2[is_both]
                best_score[is_both] = predicted_classes[is_both]
                o_best_l2[o_is_both] = l2[o_is_both]
                o_best_score[o_is_both] = predicted_classes[o_is_both]
                o_best_attack[o_is_both] = adv[o_is_both]
                #if ((logits.argmax(1) == labels) if targeted else (logits.argmax(1) != labels)) & (logits.max() - model(inputs).max() >= self.confidence):
                #  break   
                T = time.time() - start_time

            #print("iteration:", iteration)
            total_number_of_iteration += iteration 
            # adjust the constant as needed
            adv_found = (best_score == labels) if targeted else ((best_score != labels) * (best_score != -1))
            upper_bound[adv_found] = torch.min(upper_bound[adv_found], CONST[adv_found])
            adv_not_found = ~adv_found
            lower_bound[adv_not_found] = torch.max(lower_bound[adv_not_found], CONST[adv_not_found])
            is_smaller = upper_bound < 1e9
            CONST[is_smaller] = (lower_bound[is_smaller] + upper_bound[is_smaller]) / 2
            CONST[(~is_smaller) * adv_not_found] *= 10

        T = time.time() - start_time
        print(T)
        print("total_number_of_iteration", total_number_of_iteration)

        if self.quantize:
            adv_found = o_best_score != -1
            o_best_attack[adv_found] = self._quantize(model, inputs[adv_found], o_best_attack[adv_found],
                                                      labels[adv_found], targeted=targeted)

        # return the best solution found
        return o_best_attack

    def _quantize(self, model: nn.Module, inputs: torch.Tensor, adv: torch.Tensor, labels: torch.Tensor,
                  targeted: bool = False) -> torch.Tensor:

        batch_size = inputs.shape[0]
        multiplier = 1 if targeted else -1
        delta = torch.round((adv - inputs) * 255) / 255   
        delta.requires_grad_(True)
        logits = model(inputs + delta)
        is_adv = (logits.argmax(1) == labels) if targeted else (logits.argmax(1) != labels)
        i = 0
        while not is_adv.all() and i < 100:
            loss = F.cross_entropy(logits, labels, reduction='sum')
            grad = autograd.grad(loss, delta)[0].view(batch_size, -1)
            order = grad.abs().max(1, keepdim=True)[0]
            direction = (grad / order).int().float()
            direction.mul_(1 - is_adv.float().unsqueeze(1))
            delta.data.view(batch_size, -1).sub_(multiplier * direction / 255)

            logits = model(inputs + delta)
            is_adv = (logits.argmax(1) == labels) if targeted else (logits.argmax(1) != labels)
            i += 1

        delta.detach_()
        if not is_adv.all():
            delta.data[~is_adv].copy_(torch.round((adv[~is_adv] - inputs[~is_adv]) * 255) / 255)

        return inputs + delta

In [ ]:
cwattacker_original = CarliniWagnerL2_original( device=device,
                              abort_early= False,#True,
                              image_constraints=(0, 1),
                              max_iterations = 1000,
                              num_classes=10,
                              confidence = 10)

In [ ]:
from typing import Tuple, Optional
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim


class CarliniWagnerL2:

    def __init__(self,
                 image_constraints: Tuple[float, float],
                 num_classes: int,
                 #max_iterations: int,
                 confidence: float,
                 abort_early: bool ,#True,
                 #confidence: float = 0,
                 max_iterations: int,
                 learning_rate: float = 0.01,
                 search_steps: int = 9,
                 #max_iterations: int = 1000,
                 initial_const: float = 0.001,
                 quantize: bool = False,
                 device: torch.device = torch.device('cpu'),
                 callback: Optional = None) -> None:

        self.confidence = confidence
        self.learning_rate = learning_rate

        self.binary_search_steps = search_steps
        self.max_iterations = max_iterations
        self.abort_early = abort_early
        self.initial_const = initial_const
        self.num_classes = num_classes

        self.repeat = self.binary_search_steps >= 10

        self.boxmin = image_constraints[0]
        self.boxmax = image_constraints[1]
        self.boxmul = (self.boxmax - self.boxmin) / 2
        self.boxplus = (self.boxmin + self.boxmax) / 2
        self.quantize = quantize

        self.device = device
        self.callback = callback
        self.log_interval = 10

    @staticmethod
    def _arctanh(x: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
        x *= (1. - eps)
        return (torch.log((1 + x) / (1 - x))) * 0.5

    def _step(self, model: nn.Module, optimizer: optim.Optimizer, inputs: torch.Tensor, tinputs: torch.Tensor,
              modifier: torch.Tensor, labels: torch.Tensor, labels_infhot: torch.Tensor, targeted: bool,
              const: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:

        batch_size = inputs.shape[0]
        adv_input = torch.tanh(tinputs + modifier) * self.boxmul + self.boxplus # ************************************** 

        l2 = (adv_input - inputs).view(batch_size, -1).pow(2).sum(1)


        logits = model(adv_input)
        #print(logits[labels])  #

        real = logits.gather(1, labels.unsqueeze(1)).squeeze(1)
        other = (logits - labels_infhot).max(1)[0]
        if targeted:
            # if targeted, optimize for making the other class most likely
            logit_dists = torch.clamp(other - real + self.confidence, min=0)
        else:
            # if non-targeted, optimize for making this class least likely.
            logit_dists = torch.clamp(real - other + self.confidence, min=0)

        logit_loss = (const * logit_dists).sum()
        l2_loss = l2.sum()
        loss = logit_loss + l2_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        return adv_input.detach(), logits.detach(), l2.detach(), logit_dists.detach(), loss.detach()

    def attack(self, model: nn.Module, inputs: torch.Tensor, labels: torch.Tensor, noise: torch.Tensor,
               targeted: bool = False) -> torch.Tensor:

        start_time = time.time()

        batch_size = inputs.shape[0]
        inputs = (noise + inputs).clamp(0,1)
        tinputs = self._arctanh((inputs - self.boxplus) / self.boxmul)  
        # set the lower and upper bounds accordingly
        lower_bound = torch.zeros(batch_size, device=self.device)
        CONST = torch.full((batch_size,), self.initial_const, device=self.device)
        upper_bound = torch.full((batch_size,), 1e10, device=self.device)

        o_best_l2 = torch.full((batch_size,), 1e10, device=self.device)
        o_best_score = torch.full((batch_size,), -1, dtype=torch.long, device=self.device)
        o_best_attack = inputs.clone()

        # setup the target variable, we need it to be in one-hot form for the loss function
        labels_onehot = torch.zeros(labels.size(0), self.num_classes, device=self.device)
        labels_onehot.scatter_(1, labels.unsqueeze(1), 1)
        labels_infhot = torch.zeros_like(labels_onehot).scatter_(1, labels.unsqueeze(1), float('inf'))
        total_number_of_iteration = 0
        for outer_step in range(self.binary_search_steps):

            # setup the modifier variable, this is the variable we are optimizing over    
            modifier = torch.zeros_like(inputs, requires_grad=True)        


            # setup the optimizer
            optimizer = optim.Adam([modifier], lr=self.learning_rate, betas=(0.9, 0.999), eps=1e-8)
            best_l2 = torch.full((batch_size,), 1e10, device=self.device)
            best_score = torch.full((batch_size,), -1, dtype=torch.long, device=self.device)

            # The last iteration (if we run many steps) repeat the search once.
            if self.repeat and outer_step == (self.binary_search_steps - 1):
                CONST = upper_bound

            prev = float('inf')
            for iteration in range(self.max_iterations):
                # perform the attack
                adv, logits, l2, logit_dists, loss = self._step(model, optimizer, inputs, tinputs, modifier,
                                                                labels, labels_infhot, targeted, CONST)

                if self.callback and (iteration + 1) % self.log_interval == 0:
                    self.callback.scalar('logit_dist_{}'.format(outer_step), iteration + 1, logit_dists.mean().item())
                    self.callback.scalar('l2_norm_{}'.format(outer_step), iteration + 1, l2.sqrt().mean().item())

                # check if we should abort search if we're getting nowhere.
                if self.abort_early and iteration % (self.max_iterations // 10) == 0:
                    if loss > prev * 0.9999:
                        break
                    prev = loss

                # adjust the best result found so far
                predicted_classes = (logits - labels_onehot * self.confidence).argmax(1) if targeted else \
                    (logits + labels_onehot * self.confidence).argmax(1)

                is_adv = (predicted_classes == labels) if targeted else (predicted_classes != labels)
                is_smaller = l2 < best_l2
                o_is_smaller = l2 < o_best_l2
                is_both = is_adv * is_smaller
                o_is_both = is_adv * o_is_smaller

                best_l2[is_both] = l2[is_both]
                best_score[is_both] = predicted_classes[is_both]
                o_best_l2[o_is_both] = l2[o_is_both]
                o_best_score[o_is_both] = predicted_classes[o_is_both]
                o_best_attack[o_is_both] = adv[o_is_both]
                #if ((logits.argmax(1) == labels) if targeted else (logits.argmax(1) != labels)) & (logits.max() - model(inputs).max() >= self.confidence):
                #  break   
                T = time.time() - start_time

                #if T >= 5:
                #  break 

            #print("iteration:", iteration)
            total_number_of_iteration += iteration 
            # adjust the constant as needed
            adv_found = (best_score == labels) if targeted else ((best_score != labels) * (best_score != -1))
            upper_bound[adv_found] = torch.min(upper_bound[adv_found], CONST[adv_found])
            adv_not_found = ~adv_found
            lower_bound[adv_not_found] = torch.max(lower_bound[adv_not_found], CONST[adv_not_found])
            is_smaller = upper_bound < 1e9
            CONST[is_smaller] = (lower_bound[is_smaller] + upper_bound[is_smaller]) / 2
            CONST[(~is_smaller) * adv_not_found] *= 10

        T = time.time() - start_time
        print(T)
        print("total_number_of_iteration", total_number_of_iteration)

        if self.quantize:
            adv_found = o_best_score != -1
            o_best_attack[adv_found] = self._quantize(model, inputs[adv_found], o_best_attack[adv_found],
                                                      labels[adv_found], targeted=targeted)

        # return the best solution found
        return o_best_attack

    def _quantize(self, model: nn.Module, inputs: torch.Tensor, adv: torch.Tensor, labels: torch.Tensor,
                  targeted: bool = False) -> torch.Tensor:

        batch_size = inputs.shape[0]
        multiplier = 1 if targeted else -1
        delta = torch.round((adv - inputs) * 255) / 255   
        delta.requires_grad_(True)
        logits = model(inputs + delta)
        is_adv = (logits.argmax(1) == labels) if targeted else (logits.argmax(1) != labels)
        i = 0
        while not is_adv.all() and i < 100:
            loss = F.cross_entropy(logits, labels, reduction='sum')
            grad = autograd.grad(loss, delta)[0].view(batch_size, -1)
            order = grad.abs().max(1, keepdim=True)[0]
            direction = (grad / order).int().float()
            direction.mul_(1 - is_adv.float().unsqueeze(1))
            delta.data.view(batch_size, -1).sub_(multiplier * direction / 255)

            logits = model(inputs + delta)
            is_adv = (logits.argmax(1) == labels) if targeted else (logits.argmax(1) != labels)
            i += 1

        delta.detach_()
        if not is_adv.all():
            delta.data[~is_adv].copy_(torch.round((adv[~is_adv] - inputs[~is_adv]) * 255) / 255)

        return inputs + delta

In [ ]:
cwattacker = CarliniWagnerL2( device=device,
                              abort_early= False,#True,
                              image_constraints=(0, 1),
                              max_iterations = 30,
                              num_classes=10,
                              confidence = 10)

In [ ]:
#Define the noise
j = 0   #80, 7
i = int(m_1[j])
t = m
x = data[i].unsqueeze(0).cuda()
y = label[i].unsqueeze(0).cuda()
target = label[t].unsqueeze(0).cuda()
start_time = time.time()
adv = cwattacker_original.attack(model.cuda(), x, target, noise = torch.zeros_like(x), targeted=True) # torch.zeros_like(x)
print("--- %s seconds ---" % (time.time() - start_time))
noise = adv - x

pred = model(adv)
print(pred)
prediction = pred.data.max(1, keepdim=True)[1][0].item()
print(prediction)
perturbation = adv - x
print('L2 norm of perturbation: {}'.format(torch.norm(perturbation.flatten())))
print('Linf norm of perturbation: {}'.format(perturbation.abs().max()))

25.572916507720947
total_number_of_iteration 8991
--- 25.57530665397644 seconds ---
tensor([[-3.7536, -1.7938, -1.8174, -1.8724, -2.3211, -1.8116, -3.2697, -4.4865,
          8.2091, -1.7971]], device='cuda:0', grad_fn=<AddmmBackward>)
8
L2 norm of perturbation: 2.4804584980010986
Linf norm of perturbation: 0.6091643571853638


In [ ]:
### zero initialization ###
import time
corr = 0
num_7 = 0
S = 0
P = 0
L2= Linf=T= 0
t = m
for i in range(100):  #len(m_1)
  x = data[int(m_1[i])].unsqueeze(0).cuda()
  y = label[int(m_1[i])].unsqueeze(0).cuda()
  target = label[t].unsqueeze_(0).cuda()
  start_time = time.time()
  #adv_ = targeted_fgsm(model, x, target, eps = 0.001)

  adv_ = cwattacker.attack(model.cuda(), x, target, noise = noise, targeted=True) #torch.zeros_like(x)
  #adv_ = x + noise
  T += time.time() - start_time

  print("--- %s seconds ---" % (time.time() - start_time))
  pred = model(adv_)  
  prediction = pred.data.max(1, keepdim=True)[1][0].item()
  print(prediction)
  perturbation = adv_ - x
  print('L2 norm of perturbation: {}'.format(torch.norm(perturbation.flatten())))
  print('Linf norm of perturbation: {}'.format(perturbation.abs().max()))
  if prediction==y:
      corr+=1
  print("Accuracy: %.2f" % (float(corr/(i+1)*100)), i)
  S += ssim(x[0][0].detach().cpu().numpy(), adv_[0][0].detach().cpu().numpy())   
  P += psnr(x[0].detach().cpu().numpy(), adv_[0].detach().cpu().numpy())
  L2 += torch.norm(perturbation.flatten())
  Linf += perturbation.abs().max()

#plt.imshow(-x[0][0].detach().cpu().numpy()+ adv_[0][0].detach().cpu().numpy())
print("SSIM = ", S/len(m_1))
print("PSNR= ", P/len(m_1))
print("L2 = ", L2/len(m_1))
print("Linf = ", Linf/len(m_1))
print("Runtime = ", T/len(m_1))

0.7854535579681396
total_number_of_iteration 261
--- 0.7869045734405518 seconds ---
8
L2 norm of perturbation: 2.480461359024048
Linf norm of perturbation: 0.6091642379760742
Accuracy: 0.00 0
0.7858092784881592
total_number_of_iteration 261
--- 0.7872891426086426 seconds ---
8
L2 norm of perturbation: 2.999565839767456
Linf norm of perturbation: 0.7007287740707397
Accuracy: 0.00 1
0.7775523662567139
total_number_of_iteration 261
--- 0.7788550853729248 seconds ---
8
L2 norm of perturbation: 2.300217628479004
Linf norm of perturbation: 0.6091643571853638
Accuracy: 0.00 2
0.781790018081665
total_number_of_iteration 261
--- 0.7822515964508057 seconds ---
1
L2 norm of perturbation: 2.25095534324646
Linf norm of perturbation: 0.5476552248001099
Accuracy: 25.00 3
0.7727084159851074
total_number_of_iteration 261
--- 0.7740511894226074 seconds ---
1
L2 norm of perturbation: 2.313873291015625
Linf norm of perturbation: 0.6091643571853638
Accuracy: 40.00 4
0.7921380996704102
total_number_of_itera

In [ ]:
### zero initialization ###
import time
corr = 0
num_7 = 0
S = 0
P = 0
L2= Linf=T= 0
t = m
for i in range(len(m_1)):
  x = data[int(m_1[i])].unsqueeze(0).cuda()
  y = label[int(m_1[i])].unsqueeze(0).cuda()
  target = label[t].unsqueeze_(0).cuda()
  start_time = time.time()
  
  adv_ = x + noise
  T += time.time() - start_time

  print("--- %s seconds ---" % (time.time() - start_time))
  pred = model(adv_)  
  prediction = pred.data.max(1, keepdim=True)[1][0].item()
  print(prediction)
  perturbation = adv_ - x
  print('L2 norm of perturbation: {}'.format(torch.norm(perturbation.flatten())))
  print('Linf norm of perturbation: {}'.format(perturbation.abs().max()))
  if prediction==y:
      corr+=1
  print("Accuracy: %.2f" % (float(corr/(i+1)*100)), i)
  S += ssim(x[0][0].detach().cpu().numpy(), adv_[0][0].detach().cpu().numpy())   
  P += psnr(x[0].detach().cpu().numpy(), adv_[0].detach().cpu().numpy())
  L2 += torch.norm(perturbation.flatten())
  Linf += perturbation.abs().max()

#plt.imshow(-x[0][0].detach().cpu().numpy()+ adv_[0][0].detach().cpu().numpy())
print("SSIM = ", S/len(m_1))
print("PSNR= ", P/len(m_1))
print("L2 = ", L2/len(m_1))
print("Linf = ", Linf/len(m_1))
print("Runtime = ", T/len(m_1))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
--- 5.316734313964844e-05 seconds ---
8
L2 norm of perturbation: 3.9445836544036865
Linf norm of perturbation: 0.2500000596046448
Accuracy: 0.00 1
--- 2.4080276489257812e-05 seconds ---
8
L2 norm of perturbation: 3.9445834159851074
Linf norm of perturbation: 0.25
Accuracy: 0.00 2
--- 2.0742416381835938e-05 seconds ---
8
L2 norm of perturbation: 3.9445834159851074
Linf norm of perturbation: 0.2500000298023224
Accuracy: 0.00 3
--- 1.6689300537109375e-05 seconds ---
8
L2 norm of perturbation: 3.9445834159851074
Linf norm of perturbation: 0.2500000298023224
Accuracy: 0.00 4
--- 1.7642974853515625e-05 seconds ---
8
L2 norm of perturbation: 3.9445834159851074
Linf norm of perturbation: 0.2500000596046448
Accuracy: 0.00 5
--- 1.5735626220703125e-05 seconds ---
8
L2 norm of perturbation: 3.9445834159851074
Linf norm of perturbation: 0.2500000596046448
Accuracy: 0.00 6
--- 2.0265579223632812e-05 seconds ---
8
L2 norm 

In [ ]:
#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

Mon Jan 11 19:18:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    36W /  70W |   1005MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------